In [ ]:
import tqdm
import json
import random
import itertools
import pandas as pd
from typing import List
from collections import Counter, OrderedDict

# Загрузка данных

In [ ]:
# Downloading data from github
!git clone https://github.com/dialogue-evaluation/SemSketches.git
%cd SemSketches

fatal: destination path 'SemSketches' already exists and is not an empty directory.
/content/SemSketches


In [ ]:
!ls data

dev  task_2  trial


Для начала, обратимся к демонстрационному набору данных

In [ ]:
CONTEXTS_TRIAL = "data/trial/contexts_trial.data"
SKETCHES_TRIAL = "data/trial/sketches_trial.data"
TRIAL = "data/trial/trial.gold"

Пример скетча

In [ ]:
sketches_dfs = dict()

sketches_trial_data = json.load(open(SKETCHES_TRIAL))
for sketch_name, sketch_columns_di in sketches_trial_data.items():
    colnames = [None] * len(sketch_columns_di)
    columns = [None] * len(sketch_columns_di)
    for colname, (col_ix, col_cells) in sketch_columns_di.items():
        colnames[col_ix] = colname
        columns[col_ix] = col_cells
    columns_ordered_dict = OrderedDict(zip(colnames, columns))
    sketches_dfs[sketch_name] = pd.DataFrame(columns_ordered_dict)

print('\n'.join(str(sketches_dfs).splitlines()[:12]))

{'trial.sketch.rus.0':     Sphere_Special                 Time  ...     ContrAgent Ch_Evaluation
0          в карты            в детстве  ...       с детьми       классно
1        в шахматы  на большой перемене  ...  с мальчишками       неплохо
2         в футбол           по вечерам  ...    с читателем       недурно
3  в азартные игры          каждый день  ...      с собакой   превосходно
4           в игры             допоздна  ...        с сыном      прилично
5         в прятки          в молодости  ...     с ребятами  замечательно

[6 rows x 6 columns], 'trial.sketch.rus.1':                      Locative_FinalPoint  ... Ch_EvaluationOfHumanTemperAndActivity
0                                к людям  ...                           скептически
1                           к литературе  ...                            критически
2                               к стихам  ...                            философски


Предикаты с контекстами

In [ ]:
# Table with context "name", predicate (to be compared with sketch), 
# starting index of the predicate in the sentence,  starting index and sentence itself.
contexts_trial_df = pd.read_json(CONTEXTS_TRIAL).transpose()
contexts_trial_df.head()

,instance,start,end,sentence
trial.sent.rus.0,стать,63,68,"Старый я, песенка моя спета, и нет у меня боль..."
trial.sent.rus.1,вызвала,6,13,Но не вызвала
trial.sent.rus.2,пойдет,40,46,"А может, и вовсе поменяет конституцию и пойдет..."
trial.sent.rus.3,стану,5,10,"Я не стану отрекаться, – объяснил он"
trial.sent.rus.4,становятся,149,159,В этом случае профиль сечения ППЭ реакции зави...


Как должно выглядеть решение задачи:

In [ ]:
# Dictionary with pairs: sentence - sketch
sent2sketch = json.load(open(TRIAL))
print(sent2sketch)

{'trial.sent.rus.0': 'trial.sketch.rus.8', 'trial.sent.rus.1': 'trial.sketch.rus.4', 'trial.sent.rus.2': 'trial.sketch.rus.6', 'trial.sent.rus.3': 'trial.sketch.rus.2', 'trial.sent.rus.4': 'trial.sketch.rus.15', 'trial.sent.rus.5': 'trial.sketch.rus.8', 'trial.sent.rus.6': 'trial.sketch.rus.14', 'trial.sent.rus.7': 'trial.sketch.rus.13', 'trial.sent.rus.8': 'trial.sketch.rus.6', 'trial.sent.rus.9': 'trial.sketch.rus.16', 'trial.sent.rus.10': 'trial.sketch.rus.1', 'trial.sent.rus.11': 'trial.sketch.rus.6', 'trial.sent.rus.12': 'trial.sketch.rus.0', 'trial.sent.rus.13': 'trial.sketch.rus.17', 'trial.sent.rus.14': 'trial.sketch.rus.4', 'trial.sent.rus.15': 'trial.sketch.rus.4', 'trial.sent.rus.16': 'trial.sketch.rus.0', 'trial.sent.rus.17': 'trial.sketch.rus.14', 'trial.sent.rus.18': 'trial.sketch.rus.19', 'trial.sent.rus.19': 'trial.sketch.rus.19', 'trial.sent.rus.20': 'trial.sketch.rus.19', 'trial.sent.rus.21': 'trial.sketch.rus.15', 'trial.sent.rus.22': 'trial.sketch.rus.0', 'trial.sen

# Реализация алгоритма

In [ ]:
sketches_dfs['trial.sketch.rus.0']

,Sphere_Special,Time,Agent,Locative,ContrAgent,Ch_Evaluation
0,в карты,в детстве,дети,на бирже,с детьми,классно
1,в шахматы,на большой перемене,мальчишки,во дворе,с мальчишками,неплохо
2,в футбол,по вечерам,пацаны,на бильярде,с читателем,недурно
3,в азартные игры,каждый день,игроки внизу,на компьютере,с собакой,превосходно
4,в игры,допоздна,ребята,на площадке,с сыном,прилично
5,в прятки,в молодости,команда,на чужом поле,с ребятами,замечательно


Получим список всех возможных ролей с примерами:

In [ ]:
roles = {}
for sketch in sketches_dfs:
    for role in list(sketches_dfs[sketch].columns):
        if role not in roles:
            roles[role] = sketches_dfs[sketch][role][0]
roles

{'Agent': 'дети',
 'Agent_Metaphoric': 'жизнь',
 'Agent_Route': 'дорога',
 'Cause_Actant': 'все пути',
 'Ch_Emotion': 'сухо',
 'Ch_Evaluation': 'классно',
 'Ch_EvaluationOfHumanTemperAndActivity': 'скептически',
 'Ch_PhysicalStateOfObject': 'аккуратно',
 'Ch_PossibilitivityInPerceptionAndRealization': 'понятно',
 'Concurrent_Situative': 'не задумываясь',
 'ContrAgent': 'с детьми',
 'Function': 'свидетелем',
 'Instrument': 'на гитаре',
 'Locative': 'на бирже',
 'LocativeEvent_FinalPoint': 'на дуэль',
 'Locative_Distance': 'ближе',
 'Locative_FinalPoint': 'к людям',
 'Locative_InitialPoint': 'из-за стола',
 'Locative_Orientation_FinalPoint': 'прочь',
 'Locative_Route': 'вдоль стены',
 'MannerOfPositionAndMotion': 'на груди',
 'Metaphoric_FinalPoint': 'в заглавие',
 'Modality': 'вряд ли',
 'Object': 'автор',
 'Object_Situation': 'читать',
 'Object_SituationLike': 'слухи',
 'OrderInTimeAndSpace': 'потом',
 'PartComplement_SituationLike': 'с уважением',
 'Possessor': 'автор',
 'Possessor_Me

Сгенерируем маски с контекстами для предсказаний

In [ ]:
def generate_masked_contexts(sketch: str) -> List[List[str]]:
    masked_contexts = []
    for role in list(sketches_dfs[sketch].columns):
        for word in list(sketches_dfs[sketch][role]):
            if word != '':
                masked_contexts.append('[MASK] ' + word)

    return masked_contexts

In [ ]:
contexts = []
for sketch in sketches_dfs:
    contexts.append(generate_masked_contexts(sketch))

In [ ]:
# пример максированных контекстов
contexts[0]

['[MASK] в карты',
 '[MASK] в шахматы',
 '[MASK] в футбол',
 '[MASK] в азартные игры',
 '[MASK] в игры',
 '[MASK] в прятки',
 '[MASK] в детстве',
 '[MASK] на большой перемене',
 '[MASK] по вечерам',
 '[MASK] каждый день',
 '[MASK] допоздна',
 '[MASK] в молодости',
 '[MASK] дети',
 '[MASK] мальчишки',
 '[MASK] пацаны',
 '[MASK] игроки внизу',
 '[MASK] ребята',
 '[MASK] команда',
 '[MASK] на бирже',
 '[MASK] во дворе',
 '[MASK] на бильярде',
 '[MASK] на компьютере',
 '[MASK] на площадке',
 '[MASK] на чужом поле',
 '[MASK] с детьми',
 '[MASK] с мальчишками',
 '[MASK] с читателем',
 '[MASK] с собакой',
 '[MASK] с сыном',
 '[MASK] с ребятами',
 '[MASK] классно',
 '[MASK] неплохо',
 '[MASK] недурно',
 '[MASK] превосходно',
 '[MASK] прилично',
 '[MASK] замечательно']

Подгружаем RuBERT для предсказаний

In [ ]:
# set up torch `DEVICE` constant
import torch
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# Russian Bert installation 
!wget -q --show-progress -c "http://files.deeppavlov.ai/deeppavlov_data/bert/rubert_cased_L-12_H-768_A-12_pt.tar.gz"
!tar -zxf rubert_cased_L-12_H-768_A-12_pt.tar.gz

BERT_MODEL_PATH = "rubert_cased_L-12_H-768_A-12_pt"
!cp {BERT_MODEL_PATH}/bert_config.json {BERT_MODEL_PATH}/config.json

In [ ]:
!pip install transformers

In [ ]:
from transformers import BertTokenizer, BertForMaskedLM

tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_PATH)
model = BertForMaskedLM.from_pretrained(BERT_MODEL_PATH).to(DEVICE)

Some weights of the model checkpoint at rubert_cased_L-12_H-768_A-12_pt were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Нормализовываем предикаты в предложениях с помощью pymorphy2

In [ ]:
contexts_trial_df.head()

,instance,start,end,sentence
trial.sent.rus.0,стать,63,68,"Старый я, песенка моя спета, и нет у меня боль..."
trial.sent.rus.1,вызвала,6,13,Но не вызвала
trial.sent.rus.2,пойдет,40,46,"А может, и вовсе поменяет конституцию и пойдет..."
trial.sent.rus.3,стану,5,10,"Я не стану отрекаться, – объяснил он"
trial.sent.rus.4,становятся,149,159,В этом случае профиль сечения ППЭ реакции зави...


In [ ]:
!pip install pymorphy2

     |████████████████████████████████| 61kB 3.0MB/s 
     |████████████████████████████████| 8.2MB 8.0MB/s 


In [ ]:
from pymorphy2 import MorphAnalyzer

morph = MorphAnalyzer()

def tokenize_word(word: str, morph=morph) -> str:
  normalized = morph.parse(word)[0].normal_form
  return normalized

In [ ]:
contexts_trial_df['normalised_instance'] = contexts_trial_df.instance.apply(tokenize_word)
contexts_trial_df.head()

,instance,start,end,sentence,normalised_instance
trial.sent.rus.0,стать,63,68,"Старый я, песенка моя спета, и нет у меня боль...",стать
trial.sent.rus.1,вызвала,6,13,Но не вызвала,вызвать
trial.sent.rus.2,пойдет,40,46,"А может, и вовсе поменяет конституцию и пойдет...",пойти
trial.sent.rus.3,стану,5,10,"Я не стану отрекаться, – объяснил он",стать
trial.sent.rus.4,становятся,149,159,В этом случае профиль сечения ППЭ реакции зави...,становиться


Получаем список кандидатов для каждого скетча и сопоставляем с предикатами в предложениях

In [ ]:
def predict_verb_for_context(context: str) -> List[str]:
  context_ids = tokenizer(context, padding=True, return_tensors="pt")["input_ids"]
  context_tensor = torch.LongTensor(context_ids).to(DEVICE)
  outputs = model(context_tensor)
  mask_token_indices = torch.where(context_tensor == tokenizer.mask_token_id)
  token_logits = outputs['logits']
  indices = torch.topk(token_logits[mask_token_indices],100).indices
  decoded = list(map(tokenizer.convert_ids_to_tokens, indices))
  candidates = []
  for predicted_words in decoded:
    for cand in predicted_words:
      parse_word = morph.parse(cand)[0]
      if parse_word.tag.POS=='VERB':
        candidates.append(parse_word.normal_form)
  return candidates

Что получается на примере демонстрационных данных:

In [ ]:
contexts[5]

['[MASK] мусор',
 '[MASK] гроб',
 '[MASK] ведро',
 '[MASK] тело',
 '[MASK] сор',
 '[MASK] мебель',
 '[MASK] на обложку',
 '[MASK] на помойку',
 '[MASK] за пределы текста',
 '[MASK] на улицу',
 '[MASK] во двор',
 '[MASK] на свежий воздух',
 '[MASK] из дома',
 '[MASK] из мавзолея',
 '[MASK] из избы',
 '[MASK] из квартиры',
 '[MASK] с поля боя',
 '[MASK] из огня',
 '[MASK] в заглавие',
 '[MASK] в заголовок',
 '[MASK] в название',
 '[MASK] в эпиграф',
 '[MASK] в подзаголовок',
 '[MASK] за рамки воспоминаний',
 '[MASK] санитары',
 '[MASK] грузчики',
 '[MASK] служитель катка',
 '[MASK] дворник',
 '[MASK] добрый конь',
 '[MASK] воры',
 '[MASK] вперед',
 '[MASK] поочередно',
 '[MASK] только по воскресеньям',
 '[MASK] по утрам',
 '[MASK] каждый день']

Предсказания модели для шаблонов:

In [ ]:
predict_verb_for_context(contexts[5])

['умереть',
 'сохраниться',
 'быть',
 'иметься',
 'скончаться',
 'быть',
 'сохраниться',
 'иметь',
 'иметься',
 'прощать',
 'иметь',
 'сохраниться',
 'быть',
 'быть',
 'иметься',
 'использоваться',
 'сохраниться',
 'выйти',
 'попасть',
 'попасть',
 'поместить',
 'попасть',
 'поступить',
 'выйти',
 'войти',
 'вернуться',
 'попасть',
 'перенести',
 'выйти',
 'работать',
 'играть',
 'выйти',
 'говорить',
 'играть',
 'работать',
 'выйти',
 'выйти',
 'выходить',
 'выходить',
 'выйти',
 'выходить',
 'выйти',
 'выйти',
 'выйти',
 'выступать',
 'выйти',
 'выходить',
 'вернуться',
 'выходить',
 'выходить',
 'выйти',
 'выходить',
 'выйти',
 'вернуться',
 'выйти',
 'выступать',
 'впадать',
 'выйти',
 'выйти',
 'входить',
 'выходить',
 'выходить',
 'выйти',
 'выходить',
 'выйти',
 'входить',
 'выходить',
 'входить',
 'вернуться',
 'въехать',
 'выйти',
 'идти',
 'выйти',
 'зайти',
 'врезаться',
 'жить',
 'заходить',
 'выходить',
 'войти',
 'войти',
 'смотреть',
 'выйти',
 'уйти',
 'выйти',
 'выходи

Самый частый шаблон:

In [ ]:
def most_common(lst):
    return max(set(lst), key=lst.count)

most_common(predict_verb_for_context(contexts[5]))

'выйти'

# Получаем результат для тестовых данных

In [ ]:
CONTEXTS_DEV = "data/dev/contexts_dev.data"
SKETCHES_DEV = "data/dev/sketches_dev.data"

Загружаются скетчи

In [ ]:
sketches_dev_dfs = dict()

sketches_dev_data = json.load(open(SKETCHES_DEV))
for sketch_name, sketch_columns_di in sketches_dev_data.items():
    colnames = [None] * len(sketch_columns_di)
    columns = [None] * len(sketch_columns_di)
    for colname, (col_ix, col_cells) in sketch_columns_di.items():
        colnames[col_ix] = colname
        columns[col_ix] = col_cells
    columns_ordered_dict = OrderedDict(zip(colnames, columns))
    sketches_dev_dfs[sketch_name] = pd.DataFrame(columns_ordered_dict)

print('\n'.join(str(sketches_dev_dfs).splitlines()[:12]))

{'dev.sketch.rus.0':          Object_Situation  ...  DegreeNumerative
0             число газет  ...   примерно на 60%
1    количество библиотек  ...          на треть
2     численность занятых  ...  на 53,7 процента
3                   объем  ...      один миллион
4        посевная площадь  ...             людей
5  доля голосов епископов  ...                  
6   расстояние между нами  ...                  
7                    риск  ...                  

[8 rows x 6 columns], 'dev.sketch.rus.1':          Locative_Route  ... Locative_Orientation_FinalPoint
0  вдоль большой дороги  ...                       навстречу


Загружаются контексты:

In [ ]:
contexts_dev_df = pd.read_json(CONTEXTS_DEV).transpose()
contexts_dev_df.head()

,instance,start,end,sentence
dev.sent.rus.0,отбирать,169,177,"Люди, производящие очень дорогостоящее оборудо..."
dev.sent.rus.1,пожал,44,49,Он не спеша подошел к полковнику Эмсуорту и по...
dev.sent.rus.2,обратил,289,296,Докладчик также представил отчет о рассмотрени...
dev.sent.rus.3,остановился,32,43,"Но, пройдя несколько шагов, остановился и..."
dev.sent.rus.4,выделил,80,87,"Чтобы предотвратить это, Центральный фонд реаг..."


In [ ]:
sketches_dev_dfs['dev.sketch.rus.0']

,Object_Situation,DegreeIntensity,Time,DegreeMultiplicative,Object,DegreeNumerative
0,число газет,значительно,в посткризисный период,вдвое,государственные расходы,примерно на 60%
1,количество библиотек,резко,постоянно,в разы,факторные издержки производства,на треть
2,численность занятых,существенно,одновременно,,население,"на 53,7 процента"
3,объем,ничуть,к концу века,,налогооблагаемый доход,один миллион
4,посевная площадь,сильно,с годами,,сумма нашего долга,людей
5,доля голосов епископов,чуть,за последние полвека,,тиражи поэтических книг,
6,расстояние между нами,немного,до 500 г,,срок,
7,риск,и более,на какое-то время,,налоги,


Выведем роли, соответсвующие dev данным

In [ ]:
roles = {}
for sketch in sketches_dev_dfs:
    for role in list(sketches_dev_dfs[sketch].columns):
        if role not in roles:
            roles[role] = sketches_dev_dfs[sketch][role][0]
roles

{'Addition': 'также',
 'AdditiveDegree': 'еще',
 'AdditiveQuantity': 'еще',
 'Addressee': 'врагу',
 'Addressee_Metaphoric': 'своим глазам',
 'Agent': 'коровы',
 'Agent_Device': 'поезд',
 'Agent_Metaphoric': 'годы',
 'Agent_Route': 'улица',
 'AttachedProperty': 'в союзники',
 'BehalfOfEntity': 'от первого лица',
 'BeneMalefactive': 'русскому читателю',
 'Causator': 'мать',
 'Cause': 'по какой-то причине',
 'Cause_Actant': 'жизнью',
 'Cause_From': 'от страха',
 'ChRelation_Grade': 'на три',
 'Ch_Ability': 'проницательно',
 'Ch_AvailabilityForUse': 'свободно',
 'Ch_ByLanguage': 'на русском языке',
 'Ch_Emotion': 'нежно',
 'Ch_Evaluation': 'удачно',
 'Ch_EvaluationOfHumanTemperAndActivity': 'от души',
 'Ch_Exactness': 'неточно',
 'Ch_Information': 'подробно',
 'Ch_NotabilityDistribution': 'торжественно',
 'Ch_Parameter_Density': 'неплотно',
 'Ch_Parameter_Price': 'дорого',
 'Ch_Parameter_Speed': 'быстро',
 'Ch_Perceptibility': 'резко',
 'Ch_PerceptionAndViews': 'радикально',
 'Ch_Phys_Form

In [ ]:
contexts_dev_df['normalised_instance'] = contexts_dev_df.instance.apply(tokenize_word)
contexts_dev_df.head()

,instance,start,end,sentence,normalised_instance
dev.sent.rus.0,отбирать,169,177,"Люди, производящие очень дорогостоящее оборудо...",отбирать
dev.sent.rus.1,пожал,44,49,Он не спеша подошел к полковнику Эмсуорту и по...,пожать
dev.sent.rus.2,обратил,289,296,Докладчик также представил отчет о рассмотрени...,обратить
dev.sent.rus.3,остановился,32,43,"Но, пройдя несколько шагов, остановился и...",остановиться
dev.sent.rus.4,выделил,80,87,"Чтобы предотвратить это, Центральный фонд реаг...",выделить


Сгенерируем маски 

In [ ]:
# пытаемся сделать контексты вида Agent + MASK + другая роль
def generate_masked_contexts_dev(sketch: str) -> List[List[str]]:
    agents = []
    other_roles = []
    for role in list(sketches_dev_dfs[sketch].columns):
        for word in list(sketches_dev_dfs[sketch][role]):
            if word != '':
                if role.split('_')[0] == 'Agent':
                    agents.append(word)
                else:
                    other_roles.append(word)
    if agents == []:
        agents = ['он', 'они']
    role_pairs = list(itertools.product(agents, other_roles))
    masked_contexts = [pair[0] + ' [MASK] ' + pair[1] for pair in role_pairs]
    return masked_contexts

In [ ]:
# делаем попарные контексты
def generate_masked_contexts_dev_pairs(sketch: str) -> List[List[str]]:
    masked_contexts = []
    for role in list(sketches_dev_dfs[sketch].columns):
        for word in list(sketches_dev_dfs[sketch][role]):
            if word != '' and role.split('_')[0] != 'Agent':
            #if word != '':
                masked_contexts.append('[MASK] ' + word)
                masked_contexts.append(word + ' [MASK]')
    return masked_contexts

In [ ]:
contexts_dev = []
for sketch in sketches_dev_dfs:
    contexts_dev.append(generate_masked_contexts_dev(sketch) + generate_masked_contexts_dev_pairs(sketch))

In [ ]:
contexts_dev = []
for sketch in sketches_dev_dfs:
    contexts_dev.append(generate_masked_contexts_dev_pairs(sketch))

In [ ]:
contexts_dev[0]

['он [MASK] число газет',
 'он [MASK] количество библиотек',
 'он [MASK] численность занятых',
 'он [MASK] объем',
 'он [MASK] посевная площадь',
 'он [MASK] доля голосов епископов',
 'он [MASK] расстояние между нами',
 'он [MASK] риск',
 'он [MASK] значительно',
 'он [MASK] резко',
 'он [MASK] существенно',
 'он [MASK] ничуть',
 'он [MASK] сильно',
 'он [MASK] чуть',
 'он [MASK] немного',
 'он [MASK] и более',
 'он [MASK] в посткризисный период',
 'он [MASK] постоянно',
 'он [MASK] одновременно',
 'он [MASK] к концу века',
 'он [MASK] с годами',
 'он [MASK] за последние полвека',
 'он [MASK] до 500 г',
 'он [MASK] на какое-то время',
 'он [MASK] вдвое',
 'он [MASK] в разы',
 'он [MASK] государственные расходы',
 'он [MASK] факторные издержки производства',
 'он [MASK] население',
 'он [MASK] налогооблагаемый доход',
 'он [MASK] сумма нашего долга',
 'он [MASK] тиражи поэтических книг',
 'он [MASK] срок',
 'он [MASK] налоги',
 'он [MASK] примерно на 60%',
 'он [MASK] на треть',
 'он [M

In [ ]:
def predict_verb_for_context(context: str) -> List[str]:
  context_ids = tokenizer(context, padding=True, return_tensors="pt")["input_ids"]
  context_tensor = torch.LongTensor(context_ids).to(DEVICE)
  outputs = model(context_tensor)
  mask_token_indices = torch.where(context_tensor == tokenizer.mask_token_id)
  token_logits = outputs['logits']
  indices = torch.topk(token_logits[mask_token_indices],20).indices
  decoded = list(map(tokenizer.convert_ids_to_tokens, indices))
  candidates = []
  for predicted_words in decoded:
    for cand in predicted_words:
      parse_word = morph.parse(cand)[0]
      if parse_word.tag.POS == 'VERB':
        candidates.append(parse_word.normal_form)
  return candidates

In [ ]:
def most_common(lst):
    return max(set(lst), key=lst.count)

Пример предсказания для скетча

In [ ]:
contexts_dev[3]

['[MASK] мимо',
 'мимо [MASK]',
 '[MASK] путь',
 'путь [MASK]',
 '[MASK] по коридору',
 'по коридору [MASK]',
 '[MASK] по улице',
 'по улице [MASK]',
 '[MASK] насквозь',
 'насквозь [MASK]',
 '[MASK] мимо школы',
 'мимо школы [MASK]',
 '[MASK] вдоль китайской границы',
 'вдоль китайской границы [MASK]',
 '[MASK] закрытые дома',
 'закрытые дома [MASK]',
 '[MASK] на кухню',
 'на кухню [MASK]',
 '[MASK] в комнату',
 'в комнату [MASK]',
 '[MASK] в кабинет',
 'в кабинет [MASK]',
 '[MASK] в зал',
 'в зал [MASK]',
 '[MASK] в ванную',
 'в ванную [MASK]',
 '[MASK] в спальню',
 'в спальню [MASK]',
 '[MASK] до самого конца',
 'до самого конца [MASK]',
 '[MASK] к двери',
 'к двери [MASK]',
 '[MASK] дважды',
 'дважды [MASK]',
 '[MASK] как раз',
 'как раз [MASK]',
 '[MASK] еще раз',
 'еще раз [MASK]',
 '[MASK] скоро',
 'скоро [MASK]',
 '[MASK] изредка',
 'изредка [MASK]',
 '[MASK] только что',
 'только что [MASK]',
 '[MASK] ежедневно',
 'ежедневно [MASK]',
 '[MASK] как-то',
 'как-то [MASK]',
 '[MASK]

In [ ]:
dict(Counter(predict_verb_for_context(contexts_dev[3])))

{'##рить': 2,
 '##ть': 2,
 'быть': 24,
 'вернуться': 4,
 'взять': 1,
 'войти': 3,
 'впадать': 1,
 'встречаться': 1,
 'входить': 10,
 'выйти': 1,
 'выпускаться': 1,
 'выходить': 1,
 'жить': 1,
 'использоваться': 1,
 'находиться': 3,
 'обновляться': 1,
 'погибнуть': 1,
 'получаться': 1,
 'получиться': 1,
 'появиться': 1,
 'проводиться': 1,
 'пройти': 2,
 'проходить': 4,
 'работать': 3,
 'располагаться': 1,
 'родиться': 3,
 'сказать': 1,
 'скончаться': 1,
 'случиться': 1,
 'стоить': 1,
 'умереть': 6,
 'учиться': 2}

In [ ]:
def get_verb_prediction(cont):
    verb_counter = dict(Counter(predict_verb_for_context(cont)))
    for verb in verb_counter:
        verb_counter[verb] /= len(cont)
    return verb_counter

In [ ]:
get_verb_prediction(contexts_dev[3])

{'##рить': 0.03125,
 '##ть': 0.03125,
 'быть': 0.375,
 'вернуться': 0.0625,
 'взять': 0.015625,
 'войти': 0.046875,
 'впадать': 0.015625,
 'встречаться': 0.015625,
 'входить': 0.15625,
 'выйти': 0.015625,
 'выпускаться': 0.015625,
 'выходить': 0.015625,
 'жить': 0.015625,
 'использоваться': 0.015625,
 'находиться': 0.046875,
 'обновляться': 0.015625,
 'погибнуть': 0.015625,
 'получаться': 0.015625,
 'получиться': 0.015625,
 'появиться': 0.015625,
 'проводиться': 0.015625,
 'пройти': 0.03125,
 'проходить': 0.0625,
 'работать': 0.046875,
 'располагаться': 0.015625,
 'родиться': 0.046875,
 'сказать': 0.015625,
 'скончаться': 0.015625,
 'случиться': 0.015625,
 'стоить': 0.015625,
 'умереть': 0.09375,
 'учиться': 0.03125}

Для каждого набора скетчей получаем предсказания:

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
sketches_pred = {}
for i in tqdm(range(len(contexts_dev))):
    sketches_pred['dev.sketch.rus.' + str(i)] = get_verb_prediction(contexts_dev[i])

Фиксируем в файл

In [ ]:
with open("data_file.json", "w") as write_file:
  json.dump(sketches_pred, write_file, ensure_ascii=False)

# Подготовка данных для загрузки 

In [ ]:
contexts_dev_df.head()

,instance,start,end,sentence,normalised_instance
dev.sent.rus.0,отбирать,169,177,"Люди, производящие очень дорогостоящее оборудо...",отбирать
dev.sent.rus.1,пожал,44,49,Он не спеша подошел к полковнику Эмсуорту и по...,пожать
dev.sent.rus.2,обратил,289,296,Докладчик также представил отчет о рассмотрени...,обратить
dev.sent.rus.3,остановился,32,43,"Но, пройдя несколько шагов, остановился и...",остановиться
dev.sent.rus.4,выделил,80,87,"Чтобы предотвратить это, Центральный фонд реаг...",выделить


Функция для соотнесения предложений со скетчами

In [ ]:
def sketch_by_verb(sketches_pred, verb):
    share = 0
    # для случая, когда глагол не был найден в скетчах, берём рандомный
    # в планах - делать умный поиск по синонимам такого глагола
    num_sketch = 'dev.sketch.rus.' + str(random.randint(0, 894))
    for sketch in sketches_pred:
        if verb in sketches_pred[sketch]:
            if sketches_pred[sketch][verb] > share:
                share = sketches_pred[sketch][verb]
                num_sketch = sketch
    return num_sketch

In [ ]:
data = {}
for sent in list(contexts_dev_df.index()):
    data[sent] = sketch_by_verb(sketches_pred, contexts_dev_df['normalized_instahce'])

In [ ]:
with open("data_file.json", "w") as write_file:
    json.dump(data, write_file)